### Create a Application where generated image by the foundation model on the Amazon S3 and the program return the link of the stored image on the S3

In [1]:
import boto3
import json
import base64
import io
from PIL import Image
import uuid


bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

model_id = "amazon.nova-canvas-v1:0"

prompt = "nathuram godse killing gandhiji and it is a event which is actually happened in the past godse should held gun in his hands and keep some distance between both of them"

body = json.dumps({
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": prompt
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "height": 1024,
        "width": 1024,
        "cfgScale": 8.0,
        "seed": 0
    }
})


accept = "application/json"
content_type = "application/json"

response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    accept=accept, 
    contentType=content_type
)

result = json.loads(response["body"].read())
base64_image = result.get("images")[0]  
image_bytes = base64.b64decode(base64_image)  

image = Image.open(io.BytesIO(image_bytes))
image.show()
id=uuid.uuid4()
object_path=f"Generated_images/{id}.png"
save_path = f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/{object_path}"  # Change this path as needed
image.save(save_path)

print(f"Image successfully saved at: {save_path}")


## Save the image to the S3


s3 = boto3.client("s3")
id = uuid.uuid4()

bucket_name="myawsimgbucket-meet"


response=s3.upload_file(f"C:/Users/MeetMavani/Desktop/Meet_Mavani_Internship/GenAI Blogs/Blog-2-Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator/{object_path}",
                        bucket_name,
                        object_path,
                        ExtraArgs={'ContentType': 'image/jpeg'}
                        )

response = s3.put_object(
            Bucket=bucket_name,
            Key=object_path,
            Body=image_bytes,
            ContentType='image/jpeg'
        )

object_url = f"https://{bucket_name}.s3.us-east-1.amazonaws.com/{object_path}"
print(object_url)
print("Link has been Generated")


ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: This request has been blocked by our content filters. Please adjust your text prompt to submit a new request.